In [70]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import io
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [71]:
SOS_token = 0
EOS_token = 1
UNK_token = 2

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS",  2:"UNK"}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [72]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def Reverse(lst): 
    return [ele for ele in reversed(lst)] 


In [73]:
def readLangs(lang1, lang2, reverse=False, reverse_lang=False):
    print("Reading lines...")

    # Read the file and split into lines  
    
#     lines1 = open(io.StringIO(uploaded['dev.en'].decode('utf-8')),encoding='utf-8').read().strip().split('\n')
#     lines2 = open(io.StringIO(uploaded['dev.hi'].decode('utf-8')),encoding='utf-8').read().strip().split('\n')
    
    lines1 = open('en_train',encoding='utf-8').read().strip().split('\n')
    lines2 = open('hi_train',encoding='utf-8').read().strip().split('\n')
  
    print("len lines1 ",len(lines1))
    print("len lines2 ",len(lines2))
    print("line1 example")
    print(lines1[0])
    print("line2 example")
    print(lines2[0])
    
    if(reverse == True):
        for i in range(0,len(lines1)):
            tempLines = lines1[i].split(' ')
            temp =  Reverse(tempLines)
#             print("Temp")
#             print(temp)
            listToStr = ' '.join([str(elem) for elem in temp]) 
#             print(listToStr)
#             print(type(lines1[i]))
#             print(type(lines2[i]))
            
    print("line1 example AGAaIN")
    print(lines1[0])
    lines = []
    for i in range(0,len(lines1)):
        lines.append(str(lines1[i])+' \t '+str(lines2[i]))

    print("len(lines) ",len(lines))
    print("lines[0] ",str(lines[0]))
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    pairs = [[s for s in l.split('\t')] for l in lines]
    print("len(pairs) ",len(pairs))
    print(pairs[0])
    
    if reverse_lang:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [74]:
MAX_LENGTH =15
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [75]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('en', 'hi', True)
print("random.choice(pairs)")
print(random.choice(pairs))
orig_pairs = []
for p in pairs:
    temp = str(p[0]).split(' ')
    temp = Reverse(temp)
    listToStr = ' '.join([str(elem) for elem in temp]) 
#     print("listToStr")
#     print(listToStr)
    listToStr = str(listToStr)
    orig_pairs.append(listToStr+'\t'+str(p[1]))
#     print(orig_pairs[-1])
print("orig_pairs")
print(orig_pairs[0])
print(random.choice(orig_pairs))



Reading lines...
len lines1  200
len lines2  200
line1 example
but you will also be safe from eye diseases .
line2 example
बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।
line1 example AGAaIN
but you will also be safe from eye diseases .
len(lines)  200
lines[0]  but you will also be safe from eye diseases . 	 बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।
len(pairs)  200
['but you will also be safe from eye diseases . ', ' बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।']
Read 200 sentence pairs
Trimmed to 71 sentence pairs
Counting words...
Counted words:
en 326
hi 346
random.choice(pairs)
['Pay attention to bodily cleanliness . ', ' शारीरिक स्वच्छता पर ध्यान दें ।']
orig_pairs
 . diseases eye from safe be also will you but	 बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।
 . also side another exercise same the Do	 यही क्रिया दूसरी तरफ भी करें ।


In [76]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [77]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size,bidirectional=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [78]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size,bidirectional=True)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [79]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] if word in lang.word2index else UNK_token for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [80]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [81]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [82]:
loss_list = []
epoch_list = []

In [83]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            
            loss_list.append(print_loss_avg)
            epoch_list.append(iter)
            print('epoch = ',epoch_list[-1],'  loss = ',loss_list[-1])

            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

#     showPlot(plot_losses)

In [84]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            try:
              if topi.item() == UNK_token:
                  decoded_words.append('<UNK>')
              if topi.item() == EOS_token:
                  decoded_words.append('<EOS>')
                  break
              else:
                  decoded_words.append(output_lang.index2word[topi.item()])
            except:
              continue
            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [85]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(orig_pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [86]:
print(output_lang.n_words)
print(input_lang.n_words)
print(hidden_size)
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 25000, print_every=1000)

346
326
256


RuntimeError: Expected hidden size (2, 1, 256), got (1, 1, 256)

In [ ]:
no_of_epoch = 25000
no_hidden_states = 256
model_type = "AttnDecoder"
model_name_dec =  model_type+"_Model_"+str(no_of_epoch)+"_"+str(no_hidden_states)
torch.save(attn_decoder1.state_dict(), model_name_dec)
# device = torch.device('cpu')
decoder_model = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)
decoder_model.load_state_dict(torch.load(model_name_dec, map_location=device))

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

In [ ]:
print(epoch_list)
print(loss_list)


In [ ]:
s = "epoch_list_"+model_name_dec+'.pkl' 
with open(s, 'wb') as f:
    pickle.dump(epoch_list, f)

s = "loss_list"+model_name_dec+'.pkl' 
with open(s, 'wb') as f:
    pickle.dump(loss_list, f)

In [ ]:
print("after loading pickles")
s = "epoch_list_"+model_name_dec+'.pkl' 
with open(s, 'wb') as f:
    mynewlist = pickle.load(f)
    print(mynewlist)

s = "loss_list"+model_name_dec+'.pkl' 
with open(s, 'wb') as f:
    mynewlist = pickle.load(f)
    print(mynewlist)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction    

def calculate_bleu(pred_trg, real_trg):
    smoothie = SmoothingFunction().method4
    score = sentence_bleu(real_trg, pred_trg, smoothing_function=smoothie)
    return score 

def calculate_Result(encoder, decoder,lcl_pairs, n=50):
    device = torch.device('cuda')

    result_value_bleu_score = []
    
    for i in range(n):
        pair = random.choice(lcl_pairs)
        if debug:
          print('>', pair[0])
          print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        if debug:
          print('<', output_sentence)
        reference = [pair[1].split()]
        if debug:
          print('--', reference)
        output_words = output_words[:-1]
        temp  = []
        for ow in output_words:
          if ow!='':
            temp.append(ow)
        output_words = temp
        target_predicted = output_words
        
        if debug:        
          print('<<', output_words)
        
        score = calculate_bleu(target_predicted,reference)
        
        if debug:
          print("---Value",score)
        
        result_value_bleu_score.append((pair[0],pair[1].split(),target_predicted,score))

    return result_value_bleu_score

In [ ]:
def readLangsTest(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines  
    
    # lines1 = open(io.StringIO(uploaded['dev.en'].decode('utf-8')),encoding='utf-8').read().strip().split('\n')
    # lines2 = open(io.StringIO(uploaded['dev.hi'].decode('utf-8')),encoding='utf-8').read().strip().split('\n')

#     lines1 = open('/content/gdrive/My Drive/nlpa/test.en',encoding='utf-8').read().strip().split('\n')
#     lines2 = open('/content/gdrive/My Drive/nlpa/test.hi',encoding='utf-8').read().strip().split('\n')

    lines1 = open('en_test',encoding='utf-8').read().strip().split('\n')
    lines2 = open('hi_test',encoding='utf-8').read().strip().split('\n')

    print("len lines1 ",len(lines1))
    print("len lines2 ",len(lines2))
    print("line2 example")
    print(lines2[0])
 
    lines = []
    for i in range(0,len(lines1)):
        lines.append(lines1[i]+' \t '+lines2[i])
    print("len(lines) ",len(lines))
    print("lines[0] ",lines[0])
    pairs = [[s for s in l.split('\t')] for l in lines]
    print("len(pairs) ",len(pairs))
    reverse = False
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
def prepareDataTest(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangsTest(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareDataTest('eng', 'hi', True)
print(random.choice(pairs))

In [ ]:
global debug
debug = 1
device = torch.device('cpu')
result_value_bleu_score = calculate_Result(encoder_model, decoder_model,pairs)
result_value_bleu_score_dict = {}
result_value_bleu_score_dict['result'] = result_value_bleu_score 
# torch.save(result_value_bleu_score_dict, train_result_data_path)
for item in result_value_bleu_score:
  if (item[3]>0):
    print(" Source Language ",item[0])
    print(" Input Target",item[1])
    print(" Output Target",item[2])
    print(" Score ",item[3])